<a href="https://colab.research.google.com/github/padmathanumoorthy/Amazon-Bedrock-And-StabilityAI-Project/blob/main/Text_To_Image_Stable_Diffusion_XL_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install PIP

In [1]:
%%sh

# create virtual python environment
python3 -m pip install pip -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.7 MB/s eta 0:00:00


# boto3 lib and other libraries Installation

In [2]:
%%sh

# install requirements
python3 -m pip install boto3 -Uq
python3 -m pip install botocore -Uq
python3 -m pip install ffmpeg -Uq
python3 -m pip install ffmpeg-python -Uq
python3 -m pip install Pillow -Uq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 16.0 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.2.0 which is incompatible.


# Authenticate With AWS Credentials

In [9]:
import os
from google.colab import userdata
aws_access_key_id = userdata.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = userdata.get('AWS_SECRET_ACCESS_KEY')

# Create Folder for Saving Images

In [4]:
%%sh

mkdir /content/generated_images/

ls /content/

generated_images
sample_data


# Amazon Bedrock - Stable Diffusion XL v1 Model ID

In [5]:
# Amazon Bedrock Model ID used throughout this notebook
# Model IDs: https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns
MODEL_ID = "stability.stable-diffusion-xl-v1"

# Image path directry instantiation

In [6]:
# Location of visual contents used throughout this notebook
GENERATED_IMAGES = "/content/generated_images"

# Generation of an image with SDXL 1.0 with Text Prompting

In [12]:
# The original AWS example code: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-diffusion-1-0-text-image.html#model-parameters-diffusion-1-0-code-example
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.

"""
The generation of an image with SDXL 1.0 with Text Prompting
"""

import base64
import io
import json
import logging
import boto3
from PIL import Image
import time

from botocore.exceptions import ClientError

# https://prompthero.com/prompt/4c85ab1c267
# size 1024x1344  seed:427166973  cfg_scale: 3.5  steps 23  model_hash : aeb7e9e689
POSITIVE_PROMPT = "photorealistic, lotr, A tiny red dragon curled up asleep inside a nest, (Soft Focus) , (f_stop 2.8) , (focal_length 50mm) macro lens f/2. 8, medieval wizard table, (pastel) colors, (cozy) morning light filtering through a nearby window, (whimsical) steam shapes, captured with a (Canon EOS R5) , highlighting (serene) comfort, medieval, dnd, rpg, 3d, 16K, 8K"
NEGATIVE_PROMPT = ""

# Cat prompt: https://civitai.com/images/6826034
# No preset style, cfg_scale: 12, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 25
# POSITIVE_PROMPT = "amazing quality, masterpiece, best quality, hyper detailed, ultra detailed, UHD, perfect anatomy, magic world, (kitten and fish:1.4), fish in the air, spell magic to get fresh fish as food,( fish jumping from magic book:1.3), energy flow, a full body of a cute kitten, kawaii, wearing witches robe, witches hat, holding magic book, magic book on one hand, spell magic, hkmagic, extremely detailed, glowneon, glowing"
# NEGATIVE_PROMPT = "FastNegativeV2, watermark, signature, worst quality, low quality, normal quality, lowres, simple background, inaccurate limb, extra fingers, fewer fingers, missing fingers, extra arms, (extra legs:1.3), inaccurate eyes, bad composition, bad anatomy, error, extra digit, fewer digits, cropped, low res, worst quality, low quality, normal quality, jpeg artifacts, extra digit, fewer digits, trademark, watermark, artist's name, username, signature, text, words"

class ImageError(Exception):
    """
    Custom exception for errors returned by SDXL 1.0.
    """

    def __init__(self, message):
        self.message = message


# Set up logging
logger = logging.getLogger(__name__)
if logger.hasHandlers():
    logger.handlers.clear()
handler = logging.StreamHandler()
logger.addHandler(handler)
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
logger.setLevel(logging.INFO)


def main():
    build_request_body(
        MODEL_ID,
        POSITIVE_PROMPT,
        NEGATIVE_PROMPT,
    )


def generate_image(model_id, body):
    """
    Generate an image using SDXL 1.0 on demand.
    Args:
        model_id (str): The model ID to use.
        body (str) : The request body to use.
    Returns:
        image_bytes (bytes): The image generated by the model.
    """

    logger.info("Generating image with SDXL model %s", model_id)

    bedrock = boto3.client(service_name="bedrock-runtime",
                           region_name='us-west-2',
                           aws_access_key_id = aws_access_key_id,
                           aws_secret_access_key = aws_secret_access_key)

    accept = "application/json"
    content_type = "application/json"

    response = bedrock.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=content_type
    )
    response_body = json.loads(response.get("body").read())
    logger.info(f"Bedrock result: {response_body['result']}")

    base64_image = response_body.get("artifacts")[0].get("base64")
    base64_bytes = base64_image.encode("ascii")
    image_bytes = base64.b64decode(base64_bytes)

    finish_reason = response_body.get("artifacts")[0].get("finishReason")

    if finish_reason == "ERROR" or finish_reason == "CONTENT_FILTERED":
        raise ImageError(f"Image generation error. Error code is {finish_reason}")

    logger.info("Successfully generated image with the SDXL 1.0 model %s", model_id)

    return image_bytes


def build_request_body(
    model_id,
    positive_prompt,
    negative_prompt,
):
    """
    Entrypoint for SDXL example.
    Args:
        model_id (str): The model ID to use.
        positive_prompt (str): The positive prompt to use.
        negative_prompt (str): The negative prompt to use.
    """

    # Build request body
    body = json.dumps(
        {
            "text_prompts": [
                {"text": positive_prompt, "weight": 1},
                {"text": negative_prompt, "weight": -1},
            ],
            "height": 1024,
            "width": 1024,
            "cfg_scale": 3.5,
            "clip_guidance_preset": "NONE",
            "sampler": "K_DPMPP_2M",
            "samples": 1,
            "seed": 427166973,
            "steps": 25,
            "style_preset": "fantasy-art",
            "model-hash" : "aeb7e9e689",
        }
    )

    # Generate and save image
    try:
        image_bytes = generate_image(model_id=model_id, body=body)
        image = Image.open(io.BytesIO(image_bytes))
        logger.info(image.show())
        logger.info(image.size)
        epoch_time = int(time.time())
        generated_image_path = f"{GENERATED_IMAGES}/image_{epoch_time}.png"
        logger.info(f"Generated image: {generated_image_path}")
        image.save(generated_image_path)
    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
    except ImageError as err:
        message = err.response["Error"]["Message"]
        logger.error("A image error occurred: %s",message)

    else:
        logger.info(f"Finished generating image with SDXL model {model_id}.")


if __name__ == "__main__":
    main()

2024-03-10 22:05:47,308 - INFO - Generating image with SDXL model stability.stable-diffusion-xl-v1
INFO:__main__:Generating image with SDXL model stability.stable-diffusion-xl-v1
2024-03-10 22:05:53,190 - INFO - Bedrock result: success
INFO:__main__:Bedrock result: success
2024-03-10 22:05:53,204 - INFO - Successfully generated image with the SDXL 1.0 model stability.stable-diffusion-xl-v1
INFO:__main__:Successfully generated image with the SDXL 1.0 model stability.stable-diffusion-xl-v1
2024-03-10 22:05:54,056 - INFO - None
INFO:__main__:None
2024-03-10 22:05:54,060 - INFO - (1024, 1024)
INFO:__main__:(1024, 1024)
2024-03-10 22:05:54,062 - INFO - Generated image: /content/generated_images/image_1710108354.png
INFO:__main__:Generated image: /content/generated_images/image_1710108354.png
2024-03-10 22:05:54,931 - INFO - Finished generating image with SDXL model stability.stable-diffusion-xl-v1.
INFO:__main__:Finished generating image with SDXL model stability.stable-diffusion-xl-v1.
